# Battle of the Neighborhoods

## IBM Data Science Coursera Capstone

In [2]:
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Import Necessary Libraries for later activities

In [1]:
import json
import requests
import random
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported')

Libraries imported


### Import table from Wikipedia to Pandas dataframe

In [49]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_zips = pd.read_html(wiki_url, na_values = ['Not assigned'])
df_toronto_zips = toronto_zips[0]

### Drop NaN values and reset the index

In [50]:
df_toronto_zips.dropna(thresh=2, inplace=True)
df_toronto_zips.reset_index(drop=True).head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Display shape of the dataframe

In [48]:
df_toronto_zips.shape

(103, 3)

### Install geocoder library

In [68]:
import geocoder
print('Geocoder installed')

Geocoder installed


In [69]:
df_toronto_zips['Postal Code'][0:5]

2    M3A
3    M4A
4    M5A
5    M6A
6    M7A
Name: Postal Code, dtype: object

### Attempted to use geocoder to pull coordinates but it did not work (I was able to get the base coordinates for the city of Toronto though)

In [105]:
city = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explore")
location = geolocator.geocode(city)

print("Toronto's latitude is",location.latitude)
print("Toronto's longitude is",location.longitude)

Toronto's latitude is 43.6534817
Toronto's longitude is -79.3839347


### Imported .csv file provided for postal codes

In [96]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a53ef0f648fe4b2985bd333edba34e63 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='CbXyX_f_INkRkzmhWM0_HzvKjTpmOYVjAsNMuI-fjM2t',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_a53ef0f648fe4b2985bd333edba34e63.get_object(Bucket='ibmcourseracapstoneproject-donotdelete-pr-fddqjokkuthc0d',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_postal_codes = pd.read_csv(body)
df_postal_codes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merging postal code dataframe with neighborhood dataframe

In [101]:
df_toronto_neighborhoods = df_toronto_zips.merge(df_postal_codes)

In [102]:
df_toronto_neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Import folium and instantiate a map of Toronto

In [111]:
import folium
latitude = location.latitude
longitude = location.longitude
map_toronto = folium.Map(location=[latitude,longitude] , zoom_start=12)
map_toronto

### Imposed red markers on the map of Toronto for each postal code

In [118]:
for lat, lng, borough, neighborhood in zip(df_toronto_neighborhoods['Latitude'], df_toronto_neighborhoods['Longitude'], df_toronto_neighborhoods['Borough'], df_toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

In [125]:
df_toronto_neighborhoods.head(3)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636


### Set-up FourSquare API using Toronto Neighborhood coordinates

In [119]:
CLIENT_ID = 'O2CP0BR0AURTH4LGAF1D3LOR2UEFDT4LLPZHLWJ0BETN4SDO' 
CLIENT_SECRET = 'XMHVQ3QEXBY43UDYGXTDV54IOSBS3R43KYAYWILH5ZISUEI3' 
VERSION = '20200616'

In [137]:
radius = 2000
limit = 10
query = df_toronto_neighborhoods.loc[1, "Postal Code"]
postal_code_latitude = df_toronto_neighborhoods.loc[0, "Latitude"]
postal_code_longitude = df_toronto_neighborhoods.loc[0, "Longitude"]
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, postal_code_latitude, postal_code_longitude, VERSION, query, radius, limit)

In [158]:
results = requests.get(url).json()

In [140]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [159]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,A&W,Fast Food Restaurant,43.760643,-79.326865
1,Highway 401 & 404 Merge,Intersection,43.768079,-79.338722


### Repurposed FourSquare lab function to pull all venues within 1500 meters of each set of coordinates

In [160]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [163]:
toronto_venues = getNearbyVenues(names=df_toronto_neighborhoods['Postal Code'],
                                   latitudes=df_toronto_neighborhoods['Latitude'],
                                   longitudes=df_toronto_neighborhoods['Longitude'])

### Found over 2000 venues across the city of Toronto

In [154]:
toronto_venues.shape

(2005, 7)

In [164]:
df_toronto_venues = toronto_venues.drop(['Postal Code Latitude', 'Postal Code Longitude'],axis=1)

In [166]:
df_toronto_venues.head(10)

,Postal Code,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,M3A,Brookbanks Park,43.751976,-79.332140,Park
3,M3A,LCBO,43.757774,-79.314257,Liquor Store
4,M3A,Tim Hortons,43.760668,-79.326368,Café
5,M3A,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
6,M3A,High Street Fish & Chips,43.745260,-79.324949,Fish & Chips Shop
7,M3A,A&W,43.760643,-79.326865,Fast Food Restaurant
8,M3A,Tim Hortons,43.752814,-79.314067,Coffee Shop
9,M3A,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy


### Used one-hot encoding to count the number of each type of venue, sorted by postal code

In [172]:
toronto_encoding = pd.get_dummies(df_toronto_venues[['Venue Category']], prefix='', prefix_sep='')
toronto_encoding['Postal Code'] = df_toronto_venues['Postal Code']
new_columns = [toronto_encoding.columns[-1]] + list(toronto_encoding.columns[:-1])
toronto_encoding = toronto_encoding[new_columns]
toronto_encoding.head()

,Postal Code,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [184]:
toronto_postal_groups = toronto_encoding.groupby('Postal Code').mean().reset_index()
toronto_postal_groups.head()

,Postal Code,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.05
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.00
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00


In [188]:
for postal_code in toronto_postal_groups['Postal Code']:
    temp_table = toronto_postal_groups[toronto_postal_groups['Postal Code'] == postal_code].T.reset_index()
    temp_table.columns = ['category', 'frequency']
    temp_table = temp_table.iloc[1:]
    temp_table['frequency'] = temp_table['frequency'].astype(float)
    temp_table = temp_table.round({'frequency':2})
    temp_table = temp_table.sort_values('frequency',ascending=False).reset_index(drop=True).head(5)

In [190]:
def most_common_venues(row, num_categories):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_categories]

### Created a dataframe of the top 7 venue categories for each postal code

In [291]:
top_categories = 7
indicators = ['st', 'nd', 'rd']

columns = ['Postal Code']
for ind in np.arange(top_categories):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

postal_venues_sorted = pd.DataFrame(columns=columns)
postal_venues_sorted['Postal Code'] = toronto_postal_groups['Postal Code']

for ind in np.arange(toronto_postal_groups.shape[0]):
    postal_venues_sorted.iloc[ind, 1:] = most_common_venues(toronto_postal_groups.iloc[ind, :], top_categories)

postal_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M1B,Restaurant,Fast Food Restaurant,Pizza Place,Zoo Exhibit,Gas Station,Coffee Shop,Paper / Office Supplies Store
1,M1C,Hotel,Park,Burger Joint,Neighborhood,Italian Restaurant,Breakfast Spot,Grocery Store
2,M1E,Bank,Pizza Place,Sports Bar,Coffee Shop,Restaurant,Discount Store,Sandwich Place
3,M1G,Coffee Shop,Pharmacy,Bank,Pizza Place,Indian Restaurant,Burger Joint,Music Store
4,M1H,Indian Restaurant,Hakka Restaurant,Coffee Shop,Clothing Store,Restaurant,Chinese Restaurant,Music Store


### K-Means clustering with 7 clusters

In [317]:
clusters = 7
toronto_clustering = toronto_postal_groups.drop('Postal Code', axis=1)
k_means = KMeans(n_clusters=clusters, random_state=0).fit(toronto_clustering)
k_means.labels_[0:7]

array([2, 1, 2, 2, 0, 2, 2], dtype=int32)

In [293]:
postal_venues_sorted.insert(0, 'Cluster Label', k_means.labels_)

In [294]:
postal_venues_sorted.head()

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,2,M1B,Restaurant,Fast Food Restaurant,Pizza Place,Zoo Exhibit,Gas Station,Coffee Shop,Paper / Office Supplies Store
1,1,M1C,Hotel,Park,Burger Joint,Neighborhood,Italian Restaurant,Breakfast Spot,Grocery Store
2,2,M1E,Bank,Pizza Place,Sports Bar,Coffee Shop,Restaurant,Discount Store,Sandwich Place
3,2,M1G,Coffee Shop,Pharmacy,Bank,Pizza Place,Indian Restaurant,Burger Joint,Music Store
4,0,M1H,Indian Restaurant,Hakka Restaurant,Coffee Shop,Clothing Store,Restaurant,Chinese Restaurant,Music Store


In [295]:
toronto_clusters = df_toronto_venues.merge(postal_venues_sorted.set_index('Postal Code'), on='Postal Code')
toronto_clusters.head()

,Postal Code,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M3A,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant,2,Pharmacy,Coffee Shop,Gas Station,Sandwich Place,Beer Store,Caribbean Restaurant,Bank
1,M3A,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course,2,Pharmacy,Coffee Shop,Gas Station,Sandwich Place,Beer Store,Caribbean Restaurant,Bank
2,M3A,Brookbanks Park,43.751976,-79.332140,Park,2,Pharmacy,Coffee Shop,Gas Station,Sandwich Place,Beer Store,Caribbean Restaurant,Bank
3,M3A,LCBO,43.757774,-79.314257,Liquor Store,2,Pharmacy,Coffee Shop,Gas Station,Sandwich Place,Beer Store,Caribbean Restaurant,Bank
4,M3A,Tim Hortons,43.760668,-79.326368,Café,2,Pharmacy,Coffee Shop,Gas Station,Sandwich Place,Beer Store,Caribbean Restaurant,Bank


## Map Clusters
#### I could not figure out how to fix the markers being based on venue categories rather than the postal codes 

In [321]:
cluster_map = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clusters['Venue Latitude'], toronto_clusters['Venue Longitude'], toronto_clusters['Postal Code'], toronto_clusters['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(cluster_map)
       
cluster_map

## Cluster 1 - Lots of International Food
#### These are definitely good places to live if you like to try new and different cuisines

In [300]:
international_food_cluster = postal_venues_sorted.loc[postal_venues_sorted['Cluster Label'] == 0]
international_food_cluster

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
4,0,M1H,Indian Restaurant,Hakka Restaurant,Coffee Shop,Clothing Store,Restaurant,Chinese Restaurant,Music Store
10,0,M1P,Chinese Restaurant,Coffee Shop,Indian Restaurant,Ice Cream Shop,Electronics Store,Japanese Restaurant,Italian Restaurant
11,0,M1R,Middle Eastern Restaurant,Burger Joint,Indian Restaurant,Mediterranean Restaurant,Breakfast Spot,Fish Market,Supermarket
12,0,M1S,Gym / Fitness Center,Chinese Restaurant,Caribbean Restaurant,Breakfast Spot,Indian Restaurant,Noodle House,Seafood Restaurant
13,0,M1T,Falafel Restaurant,Hakka Restaurant,Bank,Noodle House,Chinese Restaurant,Sandwich Place,Seafood Restaurant
14,0,M1V,Bubble Tea Shop,Chinese Restaurant,Noodle House,Dessert Shop,Coffee Shop,Event Space,Caribbean Restaurant
17,0,M2H,Chinese Restaurant,Bakery,Bank,Coffee Shop,Pharmacy,Cantonese Restaurant,Korean Restaurant
44,0,M4N,Bakery,Park,Burger Joint,Clothing Store,Flower Shop,Garden,BBQ Joint
93,0,M9A,Bank,Park,Pharmacy,Bus Line,Bus Stop,Supermarket,Liquor Store


## Cluster 2 - A Little Bit of Everything
#### These areas have a little bit of everything so they would be a good choice if you prefer to have lots of options and don't want to go too far to find anything

In [314]:
little_bit_of_everything = postal_venues_sorted.loc[postal_venues_sorted['Cluster Label'] == 1]
little_bit_of_everything

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
1,1,M1C,Hotel,Park,Burger Joint,Neighborhood,Italian Restaurant,Breakfast Spot,Grocery Store
20,1,M2L,Furniture / Home Store,Breakfast Spot,Ice Cream Shop,Sporting Goods Shop,Fish Market,Supermarket,Juice Bar
22,1,M2N,Café,Sushi Restaurant,Indonesian Restaurant,Ice Cream Shop,Movie Theater,Creperie,Seafood Restaurant
38,1,M4G,Bakery,Sports Bar,BBQ Joint,Curling Ice,Coffee Shop,Restaurant,Brewery
39,1,M4H,Gym,Afghan Restaurant,Indian Restaurant,Curling Ice,Coffee Shop,Brewery,Fish & Chips Shop
40,1,M4J,Pizza Place,Café,Gastropub,Asian Restaurant,Breakfast Spot,Ethiopian Restaurant,Beer Bar
41,1,M4K,Greek Restaurant,Italian Restaurant,Ice Cream Shop,Cosmetics Shop,Trail,Spa,Brewery
42,1,M4L,Brewery,Café,Indian Restaurant,Beach,Coffee Shop,Middle Eastern Restaurant,Ice Cream Shop
43,1,M4M,Bar,Ice Cream Shop,Middle Eastern Restaurant,Café,Pet Store,Boutique,Bookstore
51,1,M4X,Café,Japanese Restaurant,Gastropub,Farm,Taiwanese Restaurant,Jewelry Store,Italian Restaurant


## Cluster 3 - Best Places to Eat and Drink
#### These areas have so many restaurants and coffee shops so would make for great locations for a foodie

In [313]:
eat_and_drink_in_this_cluster = postal_venues_sorted.loc[postal_venues_sorted['Cluster Label'] == 2]
eat_and_drink_in_this_cluster

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,2,M1B,Restaurant,Fast Food Restaurant,Pizza Place,Zoo Exhibit,Gas Station,Coffee Shop,Paper / Office Supplies Store
2,2,M1E,Bank,Pizza Place,Sports Bar,Coffee Shop,Restaurant,Discount Store,Sandwich Place
3,2,M1G,Coffee Shop,Pharmacy,Bank,Pizza Place,Indian Restaurant,Burger Joint,Music Store
5,2,M1J,Sandwich Place,Pizza Place,Ice Cream Shop,Bank,Pharmacy,Gas Station,Beer Store
6,2,M1K,Coffee Shop,Chinese Restaurant,Fast Food Restaurant,Sports Bar,Bank,Convenience Store,Department Store
7,2,M1L,Coffee Shop,Sandwich Place,Bank,Gas Station,Burger Joint,Fast Food Restaurant,Miscellaneous Shop
15,2,M1W,Chinese Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Bank,Bakery,Pharmacy
24,2,M2R,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Bagel Shop,Bank,Discount Store,Bookstore
25,2,M3A,Pharmacy,Coffee Shop,Gas Station,Sandwich Place,Beer Store,Caribbean Restaurant,Bank
28,2,M3H,Coffee Shop,Bank,Pizza Place,Ice Cream Shop,Mediterranean Restaurant,Ski Chalet,Sushi Restaurant


## Cluster 4 - So Much Outdoor Space
#### These areas are great for those that like to exercise and be outside

In [305]:
get_outside = postal_venues_sorted.loc[postal_venues_sorted['Cluster Label'] == 3]
get_outside

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
8,3,M1M,Park,Ice Cream Shop,Coffee Shop,Sandwich Place,Beach,Sports Bar,Pub
9,3,M1N,Park,Gym Pool,General Entertainment,Restaurant,Diner,Skating Rink,Café
19,3,M2K,Park,Bank,Gas Station,Trail,Chinese Restaurant,Skating Rink,Café
31,3,M3L,Park,Plaza,Coffee Shop,Shopping Mall,Tea Room,Vietnamese Restaurant,Bank
48,3,M4T,Park,Spa,Italian Restaurant,Liquor Store,Juice Bar,Grocery Store,Tea Room
50,3,M4W,Park,Coffee Shop,Historic Site,BBQ Joint,Grocery Store,Furniture / Home Store,Italian Restaurant
68,3,M5V,Park,Harbor / Marina,Coffee Shop,Performing Arts Venue,Dog Run,Seafood Restaurant,Sculpture Garden
96,3,M9L,Park,Asian Restaurant,Electronics Store,Pharmacy,Sports Bar,Latin American Restaurant,Skating Rink


## Cluster 5 - Rural area
#### Don't want the noise and bustle of downtown? This is the area for you on the edge of the city

In [311]:
outskirts_of_town = postal_venues_sorted.loc[postal_venues_sorted['Cluster Label'] == 4]
outskirts_of_town

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
29,4,M3J,Coffee Shop,Furniture / Home Store,Restaurant,Bank,Gas Station,Massage Studio,Caribbean Restaurant
53,4,M5A,Coffee Shop,Park,Historic Site,Bakery,Pub,Performing Arts Venue,Restaurant


## Cluster 6 - "Little Italy"
#### These areas are dominated by Italian restaurants and cafes

In [312]:
little_italy_and_coffee = postal_venues_sorted.loc[postal_venues_sorted['Cluster Label'] == 5]
little_italy_and_coffee

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
18,5,M2J,Bakery,Coffee Shop,Middle Eastern Restaurant,Electronics Store,Shopping Mall,American Restaurant,Burger Joint
21,5,M2M,Korean Restaurant,Café,Coffee Shop,Thai Restaurant,Supermarket,Bubble Tea Shop,Grocery Store
23,5,M2P,Coffee Shop,Grocery Store,Burger Joint,Ramen Restaurant,Beer Store,Dog Run,Japanese Restaurant
26,5,M3B,Supermarket,Café,Mexican Restaurant,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant
27,5,M3C,Coffee Shop,Gym,Clothing Store,Middle Eastern Restaurant,Movie Theater,Café,Shopping Mall
30,5,M3K,Coffee Shop,Athletics & Sports,Turkish Restaurant,Climbing Gym,Hockey Arena,Basketball Court,Beer Store
36,5,M4C,Coffee Shop,Gastropub,Breakfast Spot,Thai Restaurant,Curling Ice,Convenience Store,Pastry Shop
37,5,M4E,Beach,Coffee Shop,Toy / Game Store,Gastropub,Japanese Restaurant,Indie Movie Theater,Mexican Restaurant
45,5,M4P,Italian Restaurant,Café,Coffee Shop,Pizza Place,Tea Room,Bakery,Supermarket
46,5,M4R,Italian Restaurant,Coffee Shop,Diner,Restaurant,Flower Shop,Café,Tea Room


## Cluster 7 - Just the Zoo
#### Not much here except the Toronto Zoo and a park

In [315]:
zoo_park_cluster = postal_venues_sorted.loc[postal_venues_sorted['Cluster Label'] == 6]
zoo_park_cluster

,Cluster Label,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
16,6,M1X,Farm,National Park,Zoo Exhibit,Doner Restaurant,Field,Fast Food Restaurant,Farmers Market
